In [1]:
import os
import shutil

import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def checkDir(path):
    
    if os.path.isdir(path):
        return True
    
    else:
        return False

def checkFile(path):
    
    if os.path.isfile(path):
        return True
    
    else:
        return False
    
def buildModel(input_shape, output_shape):
    
    model = Sequential()

    model.add(Conv2D(filters = 128,
                       kernel_size = (5, 5),
                       padding = 'same',
                       activation = 'relu',
                       input_shape = input_shape))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(filters = 256,
                       kernel_size = (5, 5),
                       padding = 'same',
                       activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(filters = 512,
                       kernel_size = (5, 5),
                       padding = 'same',
                       activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())

    model.add(Dense(output_shape, activation = 'softmax'))
    
    return model    

origin = 'utkface-new/UTKFace/'

agePath = 'age/'
genderPath = 'gender/'

pathList = [agePath, genderPath]

In [2]:
for path in pathList:
    
    if not checkDir(path):
        os.mkdir(path)

In [3]:
nameList = os.listdir(origin)

In [4]:
for name in nameList:
    
    imgPath = os.path.join(origin, name)
    splitName = name.split('_')
    
    for index in range(len(pathList)):
        
        value = splitName[index]
        path = os.path.join(pathList[index], value)
        
        if not checkDir(path):
            os.mkdir(path)
        
        filePath = os.path.join(path, name)
        
        if not checkFile(filePath):
            shutil.copy(imgPath, path)

In [5]:
imgSize = 50

In [6]:
ageList = os.listdir(agePath)

ageImg = []
ageLabel = []

for age in ageList:
    
    path = os.path.join(agePath, age)
    nameList = os.listdir(path)
    
    for name in nameList:
        
        imgPath = os.path.join(path, name)
        
        try:
            img = cv2.imread(imgPath, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (imgSize, imgSize))
            ageImg.append(img)
            ageLabel.append(int(age))
        
        except:
            pass

In [7]:
genderList = os.listdir(genderPath)

genderImg = []
genderLabel = []

for gender in genderList:
    
    path = os.path.join(genderPath, gender)
    nameList = os.listdir(path)
    
    for name in nameList:
        
        imgPath = os.path.join(path, name)
        
        try:
            img = cv2.imread(imgPath, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (imgSize, imgSize))
            genderImg.append(img)
            genderLabel.append(int(gender))
        
        except:
            pass

In [8]:
ageImg = np.array(ageImg) / 255.0
genderImg = np.array(genderImg) / 255.0

ageImg = ageImg.reshape(len(ageImg), imgSize, imgSize, 1)
genderImg = genderImg.reshape(len(genderImg), imgSize, imgSize, 1)

In [9]:
ageList = [int(i) for i in ageList]
genderList = [int(i) for i in genderList]

ageCategorical = to_categorical(ageLabel, max(ageList) + 1)
genderCategorical = to_categorical(genderLabel, len(genderList))

ageClasses = ageCategorical.shape[1]
genderClasses = genderCategorical.shape[1]

In [10]:
ageModel = buildModel((imgSize, imgSize, 1), ageClasses)
genderModel = buildModel((imgSize, imgSize, 1), genderClasses)

modelList = [ageModel, genderModel]

for model in modelList:
    model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 50, 50, 128)       3328      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 25, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 256)       819456    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 512)       3

In [11]:
ageModel.compile(optimizer = 'adam',
                 loss = 'categorical_crossentropy',
                 metrics = ['accuracy'])
genderModel.compile(optimizer = 'adam',
                    loss = 'binary_crossentropy',
                    metrics = ['accuracy'])

In [12]:
es = EarlyStopping(patience = 50)

ageCp = ModelCheckpoint('ageModel.h5', 
                        save_best_only = True)
genderCp = ModelCheckpoint('genderModel.h5', 
                           save_best_only = True)

In [13]:
ageHistory = ageModel.fit(ageImg, 
                          ageCategorical, 
                          batch_size = 512, 
                          epochs = 100, 
                          validation_split = 0.2, 
                          callbacks = [es, ageCp], 
                          shuffle = True)
genderHistory = genderModel.fit(genderImg, 
                                genderCategorical, 
                                batch_size = 512, 
                                epochs = 100, 
                                validation_split = 0.2, 
                                callbacks = [es, genderCp], 
                                shuffle = True)

Train on 18966 samples, validate on 4742 samples
Epoch 1/100
18966/18966 [==============================] - 29s 2ms/sample - loss: 3.6941 - accuracy: 0.1026 - val_loss: 8.8507 - val_accuracy: 0.0000e+00
Epoch 2/100
18966/18966 [==============================] - 24s 1ms/sample - loss: 3.4958 - accuracy: 0.1425 - val_loss: 10.8738 - val_accuracy: 0.0000e+00
Epoch 3/100
18966/18966 [==============================] - 24s 1ms/sample - loss: 3.4134 - accuracy: 0.1492 - val_loss: 12.3780 - val_accuracy: 0.0000e+00
Epoch 4/100
18966/18966 [==============================] - 24s 1ms/sample - loss: 3.2639 - accuracy: 0.1650 - val_loss: 12.1858 - val_accuracy: 0.0000e+00
Epoch 5/100
18966/18966 [==============================] - 24s 1ms/sample - loss: 3.1417 - accuracy: 0.1762 - val_loss: 13.6709 - val_accuracy: 0.0000e+00
Epoch 6/100
18966/18966 [==============================] - 24s 1ms/sample - loss: 3.0638 - accuracy: 0.1804 - val_loss: 12.7501 - val_accuracy: 0.0000e+00
Epoch 7/100
18966/1896

Epoch 2/100
18966/18966 [==============================] - 35s 2ms/sample - loss: 0.5129 - accuracy: 0.7612 - val_loss: 1.6156 - val_accuracy: 0.2581
Epoch 3/100
18966/18966 [==============================] - 35s 2ms/sample - loss: 0.4089 - accuracy: 0.8181 - val_loss: 1.7664 - val_accuracy: 0.2218
Epoch 4/100
18966/18966 [==============================] - 35s 2ms/sample - loss: 0.3743 - accuracy: 0.8353 - val_loss: 0.4354 - val_accuracy: 0.7914
Epoch 5/100
18966/18966 [==============================] - 35s 2ms/sample - loss: 0.3321 - accuracy: 0.8565 - val_loss: 0.9688 - val_accuracy: 0.5247
Epoch 6/100
18966/18966 [==============================] - 35s 2ms/sample - loss: 0.2907 - accuracy: 0.8785 - val_loss: 0.7431 - val_accuracy: 0.6455
Epoch 7/100
18966/18966 [==============================] - 35s 2ms/sample - loss: 0.2783 - accuracy: 0.8848 - val_loss: 0.4619 - val_accuracy: 0.7843
Epoch 8/100
18966/18966 [==============================] - 35s 2ms/sample - loss: 0.2727 - accuracy: